
---

# 🌐 6. Docker Networking

## A) What it is

Containers get virtual NICs + IPs so they can talk to **host**, **internet**, and **each other**.

---

## B) Network types (use when)

* **bridge** (default) → single-host, isolated apps.
* **user-defined bridge** → name-based DNS between containers (✅ recommended).
* **host** → share host network **(Linux only)**; no port-mapping overhead.
* **none** → fully isolated (tests/security).
* **overlay** → multi-host (Swarm).

---

## C) Core commands (with meaning)

```bash
docker network ls                            # list networks
docker network create mynet                  # make user-defined bridge
docker network inspect mynet                 # see subnets, members
docker network rm mynet                      # delete (if unused)
docker network connect mynet <container>     # attach container to network
docker network disconnect mynet <container>  # detach
```

---

## D) Default vs user-defined bridge

* **default `bridge`**: no DNS → use IPs only.
* **user-defined `bridge`**: DNS by container name works.

**Example**

```bash
docker network create appnet
docker run -d --name web --network appnet nginx
docker run --rm -it --network appnet busybox ping -c1 web   # resolves by name ✅
```

---

## E) Publish ports (host ↔ container)

```bash
docker run -d -p 8080:80 nginx     # host:8080 → container:80
# EXPOSE in Dockerfile is doc-only; use -p to reach from host.
```

---

## F) Host networking (Linux only)

```bash
docker run --network host nginx     # shares host net; no -p needed
```

Use for high throughput / many ports. Not available on Desktop the same way.

---

## G) Multi-network & discover IP

```bash
docker network connect appnet web2
docker inspect -f '{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' web
```

---

## H) Quick recipes

```bash
# App + DB on same user network
docker network create dbnet
docker run -d --name db  --network dbnet postgres
docker run -d --name api --network dbnet myapi     # API can use hostname "db"

# One-off curl test
docker run --rm --network dbnet curlimages/curl:8.10.1 -s http://api:8080/health
```

---

## I) Best practices (1-liners)

* Use **user-defined bridge** for app stacks (DNS, isolation).
* Keep ports closed by default; **publish only what you need**.
* Name networks per stack (e.g., `shop_net`) for clarity.
* Prefer **Compose** to define networks declaratively later.
